<a href="https://colab.research.google.com/github/tutsilianna/Automatic_Text_Processing_and_Image_Processing/blob/main/Greetings_and_Wishes_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Загрузка репозитория с проектом для получения данных**

In [ ]:
! git clone https://github.com/tutsilianna/Greetings_and_Wishes_Generator.git

Cloning into 'Greetings_and_Wishes_Generator'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 49 (delta 18), reused 21 (delta 5), pack-reused 0
Receiving objects: 100% (49/49), 12.82 MiB | 8.20 MiB/s, done.
Resolving deltas: 100% (18/18), done.


# **Подключение библиотек и гугл диска**

In [ ]:
# from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from huggingface_hub import notebook_login
from IPython.display import clear_output
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import pandas as pd
import torch
import json
import math


%matplotlib inline

In [ ]:
notebook_login()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Чтение и предобработка данных**

Прочитаем данные, удалим дубликаты, пустые значения заполним пустыми строками.

In [ ]:
data = pd.read_csv('/content/Greetings_and_Wishes_Generator/data/raw/greetings.csv', encoding='utf-8')

data.drop_duplicates(inplace=True)
data.fillna('', inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27201 entries, 0 to 95219
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   text         27201 non-null  object
 1   likes        27201 non-null  object
 2   holiday      27201 non-null  object
 3   to           27201 non-null  object
 4   description  27201 non-null  object
dtypes: object(5)
memory usage: 1.2+ MB


Удалим ненужные категории из стобца `holiday` - `Сценарии`, `Конкурсы`, `Розыгрыши`, `Извинения`, `Признания и предложения`.

In [ ]:
data = data.loc[(data['holiday'] != "Сценарии") & (data['holiday'] != "Конкурсы") & (data['holiday'] != "Розыгрыши") & (data['holiday'] != "Извинения") & (data['holiday'] != "Признания и предложения")]

data['holiday'].value_counts()

holiday
На день рождения                  9150
На праздники                      7356
СМС                               3646
Тосты                             2125
На свадьбу                         938
Ежедневные стихи                   899
Важные события                     840
По профессиям                      836
На работу и учебу                  612
Поздравления детей и для детей     200
Name: count, dtype: int64

In [ ]:
print("Всего строк в данных: " + str(len(data)))

Всего строк в данных: 26602


В данных в столбце  `text` в конце присутствует число, указывающее на количество лайков, оставленных пользователями, уберем его с помощью функции `remove_last_line_if_digits`



In [ ]:
def remove_last_line_if_digits(text):
    lines = text.split('\n')
    last_line = lines[-1]
    if last_line.isdigit():
        return '\n'.join(lines[:-1])
    else:
        return text

data['text'] = data['text'].apply(remove_last_line_if_digits)
data['text'] = data['text'].str.replace(r"\xao", ' ')

<ipython-input-6-4b596b72e315>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].apply(remove_last_line_if_digits)
<ipython-input-6-4b596b72e315>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].str.replace(r"\xao", ' ')


# **Подготовка данных к обучению**



Так как данных много и обучение проводить тяжело в рамказ Colab, то возьмем только 1000 значений с поздравлениями "На день рождения"

In [ ]:
data_ = data.copy(deep=True)
data_ = data_.loc[data_['holiday'] == 'На день рождения']
data_ = data_.loc[data_['to'] != 'Happy Birthday Wishes']
# data_ = data_[:1000]
data_ = shuffle(data_)
data_.reset_index(inplace=True)

1) Представим данные в виде:
#### **`<s>Ключевые слова: [праздник: ... ] [для кого: ... ] [описание: ... ] -> Поздравление: текст поздравления <\s>`**,

2) разделим на тренировочный и тестовый датасеты и запишем в соответствующие файлы.

In [ ]:
data_str_ = []
for i in range(len(data_)):
    item = f'<s>Ключевые слова: [Праздник: {data_.iloc[i]["holiday"]}]' + f'[Для кого: {data_.iloc[i]["to"]}]' + f'[Описание: {data_.iloc[i]["description"]}] ->\n'
    item += 'Поздравление: ' + data_.iloc[i]['text'] + '</s>'
    if i < len(data) - 1:
        item += '\n\n\n'
    data_str_.append(item)


X, y = train_test_split(data_str_, test_size=0.2, random_state=42)

with open('train.txt', 'w') as file:
    for line in X:
        file.write(line)

with open('test.txt', 'w') as file:
    for line in y:
        file.write(line)

Пример получившихся данных

In [ ]:
print(data_str_[50].strip())

<s>Ключевые слова: [Праздник: На день рождения][Для кого: Сестре][Описание: Сестре от брата] ->
Поздравление: С Днем Рождения, Любимая Сестренка,
Поздравляю тебя искренне сейчас,
Смейся ты всегда от счастья очень громко,
Пусть исходит всегда блеск из твоих глаз.

С Днем Рождения, Сестренка Дорогая,
Никогда, тебя прошу, не унывай,
Всё, о чем ты в своей жизни так мечтаешь,
При желании малейшем, получай.

В твоих мыслях, пусть, всегда будут идеи,
Ты идеи все в реальность воплоти,
Пусть любовь всегда твое сердечко греет,
Будь любима ты и искренне люби.</s>


# **Обучение `rugpt3small_bassed_on_grt2` по мануалу [RuGPT3FinetuneHF.ipynb](https://github.com/ai-forever/ru-gpts/blob/master/examples/RuGPT3FinetuneHF.ipynb)**

Установка необходимых библиотек и скачивание скрипта `run_clm.py` для обучения

In [ ]:
!pip install transformers[torch] accelerate -U
!pip install datasets evaluate
!pip install git+https://github.com/huggingface/transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 5.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
!wget https://raw.githubusercontent.com/huggingface/transformers/main/examples/pytorch/language-modeling/run_clm.py -O /content/run_clm.py

--2024-05-22 05:11:08--  https://raw.githubusercontent.com/huggingface/transformers/main/examples/pytorch/language-modeling/run_clm.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28239 (28K) [text/plain]
Saving to: ‘/content/run_clm.py’

/content/run_clm.py 100%[===================>]  27.58K  --.-KB/s    in 0s      

2024-05-22 05:11:09 (144 MB/s) - ‘/content/run_clm.py’ saved [28239/28239]



Запуск обучения модели, расположенной в `sberbank-ai/rugpt3small_based_on_gpt2`

In [ ]:
# Запуск обучения модели
!python /content/run_clm.py \
    --model_name_or_path sberbank-ai/rugpt3small_based_on_gpt2 \
    --train_file /content/train.txt \
    --validation_file /content/test.txt \
    --per_device_train_batch_size 1 \
    --per_device_eval_batch_size 1 \
    --block_size 256 \
    --dataset_config_name plain_text \
    --do_train \
    --do_eval \
    --output_dir /content/drive/MyDrive/model/essays

2024-05-22 05:12:25.038367: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-22 05:12:25.038413: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-22 05:12:25.231006: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-22 05:12:25.376502: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-22 05:12:26.395522: W tensorflow/compiler/tf2

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import torch
import math

Загрузка обученной модели и токенизатора

In [ ]:
model     = GPT2LMHeadModel.from_pretrained('/content/drive/MyDrive/model/essays').to('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = GPT2Tokenizer.from_pretrained('/content/drive/MyDrive/model/essays') #('sberbank-ai/rugpt3small_based_on_gpt2')

In [ ]:
#  Установка фиксированных зерен для воспроизводимости
np.random.seed(42)
torch.manual_seed(42)

# Перемещение модели на GPU
model.cuda()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50264, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50264, bias=False)
)

In [ ]:
# Текст для генерации
text = '<s>Ключевые слова: [Праздник: На день рождения][Для кого: Дочери][Описание: Дочери на 45 лет] ->\nПоздравление:'
input_ids = tokenizer.encode(text, return_tensors='pt')

# Генерация текста
output = model.generate(input_ids.cuda(), max_length=500, repetition_penalty=5.0, do_sample=True, top_k=5, top_p=0.95, temperature=1)

# Декодирование сгенерированных токенов в текст
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

Ключевые слова: [Праздник: На день рождения][Для кого: Дочери][Описание: Дочери на 45 лет] ->
Поздравление: Я поздравляю с днем рожденья тебя, Дочка моя. Сорок пять тебе сегодня исполняется твоей судьбе и я хочу пожелать в этот праздник всего самого прекрасного из того,чего ты сама себе пожелаешь.Пусть всегда твои сбываются желания! И пусть все то о чем мечтается будет именно так как хочешь!


# **Вычисление перплексии и косинусного сходства**

Создадим список с промтами, эталонными и сгенерированными текстами

In [ ]:
# y[0].split('->\nПоздравление: ')[-1]

promts = [ item.split('->\nПоздравление: ')[0] + '->\nПоздравление:'  for item in y]
reference_texts = [item.split('->\nПоздравление: ')[-1] for item in y]
generated_texts = []

for promt in promts:
    input_ids = tokenizer.encode(text, return_tensors='pt')

    output = model.generate(input_ids.cuda(),
                            max_length=500,
                            repetition_penalty=5.0,
                            do_sample=True,
                            top_k=5,
                            top_p=0.95,
                            temperature=1)

    generated_texts.append(tokenizer.decode(output[0], skip_special_tokens=True))

In [ ]:
generated_texts_split = [item.split('->\nПоздравление: ')[-1] for item in generated_texts]

In [ ]:
with open('generated_texts.txt', 'w') as file:
    for x in generated_texts_split:
        file.write(x)

In [ ]:
with open('generated_texts.txt', 'r') as file:
   generated_text = file.readlines()

generated_text[:2]

In [ ]:
def encode_texts(tokenizer, texts, model):
    encodings = []
    for text in texts:
        inputs = tokenizer(text, return_tensors="pt")
        inputs = inputs.to('cuda' if torch.cuda.is_available() else 'cpu')
        with torch.no_grad():
            outputs = model(**inputs, output_hidden_states=True)
        hidden_states = outputs.hidden_states[-1]                     # Последний слой скрытых состояний
        encodings.append(hidden_states[0].mean(dim=0).cpu().numpy())  # Среднее по всем токенам
    return np.vstack(encodings)

def evaluate_cosine_similarity(model, tokenizer, reference_texts, generated_texts):
    reference_encodings = encode_texts(tokenizer, reference_texts, model)
    generated_encodings = encode_texts(tokenizer, generated_texts, model)
    similarities = cosine_similarity(reference_encodings, generated_encodings)
    avg_similarity = np.mean(np.diag(similarities))
    return avg_similarity

cosine_similarity_score = evaluate_cosine_similarity(model, tokenizer, reference_texts, generated_texts_split)
print(f"Cosine Similarity: {cosine_similarity_score}")

Cosine Similarity: 0.6579082608222961


In [ ]:
# ------ вычисление перплексии ------
def evaluate_perplexity(model, eval_dataloader):
    model.eval()
    total_loss = 0.0
    nb_eval_steps = 0

    for batch in eval_dataloader:
        inputs, labels = (batch, batch)
        inputs = inputs.to('cuda' if torch.cuda.is_available() else 'cpu')
        labels = labels.to('cuda' if torch.cuda.is_available() else 'cpu')

        with torch.no_grad():
            outputs = model(inputs, labels=labels)
            loss = outputs.loss
            total_loss += loss.mean().item()
        nb_eval_steps += 1

    avg_loss = total_loss / nb_eval_steps
    perplexity = math.exp(avg_loss)
    return perplexity

test_dataset = load_dataset(tokenizer, 'path_to_test_data.txt', block_size=512)
test_dataloader = DataLoader(test_dataset, batch_size=1)

# Вычисление перплексии
perplexity = evaluate_perplexity(model, test_dataloader)
print(f"Perplexity: {perplexity}")

In [ ]:
from torch.utils.data import DataLoader, Dataset

class TextDataset(Dataset):
    def __init__(self, tokenizer, file_path, block_size=512):
        with open(file_path, 'r', encoding='utf-8') as file:
            lines = file.readlines()
        self.examples = []
        for line in lines:
            tokenized_text = tokenizer.encode(line, add_special_tokens=True, truncation=True, max_length=block_size)
            self.examples.append(tokenized_text)

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        return torch.tensor(self.examples[i], dtype=torch.long)

def load_dataset(tokenizer, file_path, block_size=512):
    return TextDataset(tokenizer, file_path, block_size)


# Путь к тестовым данным
test_data_path = '/content/test.txt'

# Загрузка и подготовка тестовых данных
test_dataset = load_dataset(tokenizer, test_data_path, block_size=512)
test_dataloader = DataLoader(test_dataset, batch_size=1)

# Функция для вычисления перплексии
def evaluate_perplexity(model, eval_dataloader):
    model.eval()
    total_loss = 0.0
    nb_eval_steps = 0

    for batch in eval_dataloader:
        inputs, labels = batch, batch
        inputs = inputs.to('cuda' if torch.cuda.is_available() else 'cpu')
        labels = labels.to('cuda' if torch.cuda.is_available() else 'cpu')

        with torch.no_grad():
            outputs = model(inputs, labels=labels)
            loss = outputs.loss
            total_loss += loss.mean().item()
        nb_eval_steps += 1

    avg_loss = total_loss / nb_eval_steps
    perplexity = math.exp(avg_loss)
    return perplexity

# Вычисление перплексии
perplexity = evaluate_perplexity(model, test_dataloader)
print(f"Perplexity: {perplexity}")

Perplexity: nan


In [ ]:
# Пути для сохранения модели и токенизатора
output_dir = '/content/drive/MyDrive/model/esesays'

# Сохранение дообученной модели
model.save_pretrained(output_dir)

# Сохранение токенизатора
tokenizer.save_pretrained(output_dir)

print(f"Модель и токенизатор сохранены в {output_dir}")

Модель и токенизатор сохранены в /content/drive/MyDrive/model/esesays


In [ ]:
output_dir = '/content/drive/MyDrive/model/esesays'

model_new     = GPT2LMHeadModel.from_pretrained(output_dir).to('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer_new = GPT2Tokenizer.from_pretrained(output_dir) #('sberbank-ai/rugpt3small_based_on_gpt2')

In [ ]:
# Текст для генерации
text = '<s>Ключевые слова: [Праздник: На день рождения][Для кого: Сыну][Описание: Сыну на 45 лет] ->\nПоздравление:'
input_ids = tokenizer_new.encode(text, return_tensors='pt')

# Генерация текста
output = model_new.generate(input_ids.cuda(), max_length=256, repetition_penalty=5.0, do_sample=True, top_k=5, top_p=0.95, temperature=1)

# Декодирование сгенерированных токенов в текст
generated_text = tokenizer_new.decode(output[0], skip_special_tokens=True)
print(generated_text)

Ключевые слова: [Праздник: На день рождения][Для кого: Сыну][Описание: Сыну на 45 лет] ->
Поздравление: Я поздравляю с днем рожденья тебя, Сын. Сорок пять тебе исполнилось сегодня и ты уже мужчина большой в душе своей сохрани тепло свое о тех днях что были когда то безвозвратно унесены тобой из прошлого чтобы не вспоминать их вновь возвращаясь к ним снова возвращаясь опять вспоминая.ты вспоминал все те дни которые могли быть лучше если бы они оставались прежними! И вот сейчас я хочу пожелать,чтобы тот миг который остался у твоей двери стал для всех возможным или почти бесконечным!!! Чтобы каждый новый год мог становиться только еще прекрасней... Пусть твой дом будет полон самых приятных сюрпризов от людей вокруг него.... пусть он станет полной чашей!!


# **Использование дообученной модели**

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

In [ ]:
path_to_model = '/content/drive/MyDrive/model/essays'

In [ ]:
model     = GPT2LMHeadModel.from_pretrained(path_to_model).to('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = GPT2Tokenizer.from_pretrained(path_to_model)

In [ ]:
holiday = 'На день рождения' # input()
to = input()
description = input()

# Текст для генерации
text = f'<s>Ключевые слова: [Праздник: {holiday}][Для кого: {to}][Описание: {description}] ->\nПоздравление:'
input_ids = tokenizer.encode(text, return_tensors='pt')

# Генерация текста
output = model.generate(input_ids.cuda(), max_length=256, repetition_penalty=5.0, do_sample=True, top_k=5, top_p=0.95, temperature=1)

# Декодирование сгенерированных токенов в текст
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)